<a href="https://colab.research.google.com/github/SushmitalKhan/Dissertation/blob/main/infer_v6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install openai
# !pip install --force-reinstall -v openai==1.55.3

In [ ]:
import openai
openai.api_key = 'read_api_key'

def count_tokens(text):
    """Returns the number of tokens in a given text."""
    return len(tokenizer.encode(text))

def truncate_list(data_list, max_tokens):
    """Truncates a list to fit within a token budget."""
    truncated_list = []
    token_count = 0
    for item in data_list:
        item_tokens = count_tokens(str(item))  # Count tokens in each item
        if token_count + item_tokens <= max_tokens:
            truncated_list.append(item)
            token_count += item_tokens
        else:
            break
    return truncated_list

def run_prompt(prompt):
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=1000,
        temperature=0.7,
        n=1, # Return only one response
        stop=None # optional stopping sequence
        # request_timeout = 60
        )
    json_response = response.choices[0].message.content.strip()

    return json_response  # Convert response to Python dictionary

In [ ]:
import pandas as pd
import re
from bs4 import BeautifulSoup
from google.colab import drive
drive.mount('/content/drive')
import json

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
powerset2023 = pd.read_csv('/content/drive/MyDrive/Dissertation/Study 1/Powerset/powerset_by_year/allActivity_2023.csv')

In [ ]:
# Select columns with less than 600 non-NaN rows
misc_columns = powerset2023.columns[powerset2023.notna().sum() < 600]

# Combine values into a new column
powerset2023['takeout1_misc_MyActivity_Search Title'] = powerset2023[misc_columns].apply(
    lambda row: ', '.join(row.dropna().astype(str)),
    axis=1)

# Drop the identified misc_columns
powerset2023 = powerset2023.drop(columns=misc_columns)


# Drop duplicate values in the specified column, keeping the first occurrence
powerset2023 = powerset2023.drop_duplicates(subset=['takeout1_chrome_MyActivity_Search Title'], keep='first')

# drop_nan_ = powerset2023.dropna(how='all')
# print(drop_nan_.count())

# powerset2023_copy = powerset2023.copy()

make prompt more dynamic --> ask for user input for inference and recommendation #

*prompt: recommend one product based on these three inferences*

make numeric instead of text --> parameterize the model --> multiply it out --> if you recommend one product based on the inferences; for example inference 1 recommendation 1, inference 1 recommendation 2 and so on

In [ ]:
#Ask to select any four column

print("\nAvailable columns:")
for i, col in enumerate(powerset2023.columns, 1):
    print(f"{i}. {col}")

# Ask the user to select any 4 columns
selected_columns = []
df_selected = pd.DataFrame()
while len(selected_columns) < 4:
    try:
        col_num = int(input(f"\nSelect column {len(selected_columns) + 1} by number (1-{len(powerset2023.columns)}): "))
        if 1 <= col_num <= len(powerset2023.columns):
            col_name = powerset2023.columns[col_num - 1]
            if col_name not in selected_columns:
                selected_columns.append(col_name)
            else:
                print("You've already selected this column. Try another.")
        else:
            print("Invalid number. Please choose a number from the list.")
    except ValueError:
        print("Invalid input. Please enter a number.")

# Extract the selected columns
powerSubset2023 = powerset2023[selected_columns]

# Print the selected DataFrame
print("\nSelected DataFrame:")
# print(powerSubset2023)

# # Optionally, save to a CSV file
save_option = input("\nWould you like to save this selection to a CSV file? (yes/no): ").strip().lower()
if save_option == "yes":
    df_selected.to_csv("/content/drive/MyDrive/Dissertation/Study 1/Inference_data/AUGUST2025/9SEP25_image_chrome_YTsearch_misc.csv", index=False)
    # print("File saved as 'selected_input_data.csv'.")

In [ ]:
#Ask to select how many inferences they wat

# Define the options
options = [3,6,9,12]

# Display the options
print("Please select a number from the options below by typing the corresponding number:\n")
for i, option in enumerate(options, 1):
    print(f"{i}. {option}")

# Get user input
user_input = input("\nEnter your selection (1-4): ")

# Validate input
if user_input.isdigit() and 1 <= int(user_input) <= 4:
    inference_no = options[int(user_input) - 1]
    print(f"\nYou selected: {inference_no}")
else:
    print("\nInvalid selection. Please enter a number between 1 and 4.")

# inference_input = int(inference_input)

for the combination of the three inferences for the combination of the three inferences
when categorizing --> ask to filter 15 to 25 percent of the data that you think are the most sensitive, interesting, or different from the rest. then categorize the rest of them --> find interesting without losing the context

In [ ]:
# Initialize list to store responses
inference_c0_json = []

# Iterate through each column in power_subset1_
for col_name, col_data in powerSubset2023[:1500].items():
  # print(col_data.dropna().tolist())
  # Define the strict JSON prompt

  prompt = f"""User wants to know what {col_data.dropna().tolist()} in {col_name} tells about them.
  1. {inference_no} inferences about the user based on their online behavior in the form "interested in X".
  2. Recommend **EXACTLY ONE** product for *EACH* inferences from step 1
  3. Recommend **EXACTLY ONE** product based on the **combination of all {inference_no} inferences** for **{col_name}**.
   **STRICT JSON OUTPUT FORMAT (No extra text or explanations, just JSON):**
   ```json
   {{
    "columns": "{col_name}",
    "inferences": [
    {', '.join([json.dumps({
      "inference": f"<output for inference {i+1}>",
      "explanation_inference": f"<explanation for inference {i+1}>",
      "recommendation": f"<recommendation for inference {i+1}>"
    }) for i in range(inference_no)])}
    ],
    "final_product_recommendation": {{
      "ONE recommendation based on ALL inferences": "<Product name and company>",
    }}
   }}
   ```
  }}"""

  print(f"🔹 Sending prompt to GPT for column: {col_name}...")
  assistant_reply = run_prompt(prompt)
  print(prompt)

  if assistant_reply:
    # Remove unwanted code block markers
    cleaned_json = re.sub(r"```json\s*|\s*```", "", assistant_reply).strip()

    try:
      # Parse JSON
      parsed_json = json.loads(cleaned_json)
      inference_c0_json.append(parsed_json)  # Store parsed data
    except json.JSONDecodeError:
      print(f"⚠️ JSON Decoding Error. Skipping entry:\n{cleaned_json[:100]}...")  # Preview problematic JSON


In [ ]:
inference_c2 = []
inference_c2_json = []

index = 0  # Keeps col_name[index] constant
col_names = list(powerSubset2023[:1500].columns)  # Convert column names to a list

while index < len(col_names):  # Outer loop for the constant column
    for next_index in range(index + 1, len(col_names)):  # Inner loop for incrementing column
        # Fetch data for the columns
        col_data_1 = powerSubset2023[:1500][col_names[index]]
        col_data_2 = powerSubset2023[:1500][col_names[next_index]]

        # Define the prompt
        prompt = f"""user wants to know what {col_data_1.dropna().tolist()} in {col_names[index]} and {col_data_2.dropna().tolist()} in {col_names[next_index]} tells about them as per the following:
        1. {user_input} inferences based on users online behavior in the form interested in X (where X is a single interest) for each column.
        2. Recommend **EXACTLY ONE** product for *EACH* inferences from step 1
        3. Recommend **EXACTLY ONE** product based on the **combination of all {inference_no} inferences** for **{col_names[index]} AND {col_names[next_index]}**.
        **STRICT JSON OUTPUT FORMAT (No extra text or explanations, just JSON):**
   ```json
   {{
    "columns": "{col_names[index]} AND {col_names[next_index]}",
    "inferences": [
    {', '.join([json.dumps({
      "inference": f"<output for inference {i+1}>",
      "explanation_inference": f"<explanation for inference {i+1}>",
      "recommendation": f"<recommendation for inference {i+1}>"
    }) for i in range(inference_no)])}
    ],
    "final_product_recommendation": {{
      "ONE recommendation based on ALL inferenceces": "<Product name and company>",
    }}
   }}
   ```
  }}

    **Strictly return a valid JSON object in this format. DO NOT include any other text or explanations.**"""

        # Run the prompt and get the response
        print(prompt)
        assistant_reply = run_prompt(prompt)
        # print(assistant_reply)

        if assistant_reply:
            # Remove unwanted code block markers
            cleaned_json = re.sub(r"```json\s*|\s*```", "", assistant_reply).strip()

            try:
                # Parse JSON
                parsed_json = json.loads(cleaned_json)
                inference_c2_json.append(parsed_json)  # Store parsed data

            except json.JSONDecodeError:
                print(f"⚠️ JSON Decoding Error. Skipping entry:\n{cleaned_json[:100]}...")  # Preview problematic JSON


    # Move to the next constant column
    index += 1

In [ ]:
inference_c3 = []
inference_c3_json = []

index = 0  # Keeps col_name[index] constant
col_names = list(powerSubset2023[:1500].columns)  # Convert column names to a list

# Outer loop keeps col_name[index] constant
while index < len(col_names):
    for next_index in range(index + 1, len(col_names) - 1):  # Iterate for combinations
        # Fetch data for the selected columns
        col_data_1 = powerSubset2023[:1500][col_names[index]]
        col_data_2 = powerSubset2023[:1500][col_names[next_index]]
        col_data_3 = powerSubset2023[:1500][col_names[next_index + 1]]

        # Define the prompt
        prompt = f"""user wants to know what {col_data_1.dropna().tolist()} in {col_names[index]} combined with {col_data_2.dropna().tolist()} in {col_names[next_index]} and {col_data_3.dropna().tolist()} in {col_names[next_index + 1]} tells about them as per the following:
        1. {user_input} inferences based on users online behavior in the form interested in X (where X is a single interest) for each column.
        2. Recommend **EXACTLY ONE** product for *EACH* inferences from step 1
        3. Recommend **EXACTLY ONE** product based on the **combination of all {inference_no} inferences** for **{col_names[index]} AND {col_names[next_index]} AND {col_names[next_index+1]}**.
        **STRICT JSON OUTPUT FORMAT (No extra text or explanations, just JSON):**
   ```json
   {{
    "columns": "{col_names[index]} AND {col_names[next_index]} AND {col_names[next_index+1]}",
    "inferences": [
    {', '.join([json.dumps({
      "inference": f"<output for inference {i+1}>",
      "explanation_inference": f"<explanation for inference {i+1}>",
      "recommendation": f"<recommendation for inference {i+1}>"
    }) for i in range(inference_no)])}
    ],
    "final_product_recommendation": {{
      "ONE recommendation based on ALL inferenceces": "<Product name and company>",
    }}
   }}
   ```
  }}

    **Strictly return a valid JSON object in this format. DO NOT include any other text or explanations.**"""

        assistant_reply = run_prompt(prompt)
        # print(assistant_reply)

        if assistant_reply:
            # Remove unwanted code block markers
            cleaned_json = re.sub(r"```json\s*|\s*```", "", assistant_reply).strip()

            try:
                # Parse JSON
                parsed_json = json.loads(cleaned_json)
                inference_c3_json.append(parsed_json)  # Store parsed data
            except json.JSONDecodeError:
                print(f"⚠️ JSON Decoding Error. Skipping entry:\n{cleaned_json[:100]}...")  # Preview problematic JSON

    index += 1


In [65]:
def save_inference_json(inference_data, inference_no, columns_used):
    """Saves the inference data to a JSON file with an automated filename."""
    base_dir = "/content/drive/MyDrive/Dissertation/Study 1/Inference_data"

    # Create a descriptive filename based on inference number and columns
    column_names = "_".join([col.split("_")[1] for col in columns_used])
    filename = f"inferences_{inference_no}_{len(columns_used)}cols_{column_names}.json"
    output_filename = f"{base_dir}/{filename}"

    with open(output_filename, "w", encoding="utf-8") as f:
        json.dump(inference_data, f, indent=4, ensure_ascii=False)

    print(f"✅ JSON saved successfully as {output_filename}")

In [ ]:
save_inference_json(inference_c2_json, inference_no, powerSubset2023.columns)